In [6]:
from keras.models import load_model
import cv2
import numpy as np
import time
import os

model = load_model("MyDataset/keras_model.h5", compile=False)
class_names = open("MyDataset/labels.txt", "r").readlines()
camera = cv2.VideoCapture(1)

sentence = [""]

while True:
    # Grab the webcamera's image.
    ret, image = camera.read()

    # Resize the raw image into (224-height,224-width) pixels
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Show the image in a window
    cv2.imshow("Webcam Image", image)

    # Make the image a numpy array and reshape it to the models input shape.
    image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

    # Normalize the image array
    image = (image / 127.5) - 1


    prediction = model.predict(image, verbose=0) # remove unncessary prints
    index = np.argmax(prediction)
    class_name = str((class_names[index])[2:]).replace("\n", "") # get similarity as a string
    confidence_score = prediction[0][index] # % similarity

    print("\033c")

    if(class_name != "background"):    # Print prediction and % similarity
        if(sentence[-1] != class_name):
            sentence.append(class_name) # need to fix    
            print(' '.join(sentence))
        print(str(np.round(confidence_score * 100))[:-2]+"% similar to", class_name)


    time.sleep(0.75) # fps captured

    keyboard_input = cv2.waitKey(1)
    if keyboard_input == 27:
        break
    # elif keyboard_input == 32: #space

camera.release()
cv2.destroyAllWindows()
# compsci 93

c
c
c
c
c
c
c
 thank you
85% similar to thank you
c
90% similar to thank you
c
c


In [7]:
from flask import Flask, render_template, Response, jsonify
import cv2
from keras.models import load_model
import numpy as np
import time
import pyttsx3
import threading

model = load_model("MyDataset/keras_model.h5", compile=False)
class_names = open("MyDataset/labels.txt", "r").readlines()
camera = cv2.VideoCapture(1)

sentence = [""]

app = Flask(__name__)
camera = cv2.VideoCapture(1)  # Access the default camera

def generate_frames():
    while True:
        success, frame = camera.read()  # Read frame from the camera
        if not success:
            break
        else:
            image = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
            image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)
            image = (image / 127.5) - 1

            prediction = model.predict(image, verbose=0)
            index = np.argmax(prediction)
            class_name = str((class_names[index])[2:]).replace("\n", "")
            confidence_score = prediction[0][index]

            if class_name != "background":
                if sentence[-1] != class_name:
                    sentence.append(class_name)
                    # if sentence[-1] != ".":
                    thread = threading.Thread(target=say, args=(class_name,))
                    thread.start()
                    
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            time.sleep(0.5)
            
def processImage(frame):
    image = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
    image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)
    image = (image / 127.5) - 1

    prediction = model.predict(image, verbose=0)
    index = np.argmax(prediction)
    class_name = str((class_names[index])[2:]).replace("\n", "")
    confidence_score = prediction[0][index]

    if class_name != "background":
        if sentence[-1] != class_name:
            sentence.append(class_name)
            # if sentence[-1] != ".":
            thread = threading.Thread(target=say, args=(class_name,))
            thread.start()

def say(text):
    speak = pyttsx3.init()
    speak.say(text)
    speak.runAndWait()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/data')
def sentence_feed():
    data = (' '.join(sentence))[1:]
    return jsonify(data=data)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\skaly\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from flask import Flask, render_template, Response, jsonify
import cv2
from keras.models import load_model
import numpy as np
import time
import pyttsx3
import threading

model = load_model("MyDataset/keras_model.h5", compile=False)
class_names = open("MyDataset/labels.txt", "r").readlines()
camera = cv2.VideoCapture(1)

sentence = [""]

app = Flask(__name__)
camera = cv2.VideoCapture(1)  # Access the default camera

def generate_frames():
    while True:
        success, frame = camera.read()  # Read frame from the camera
        if not success:
            break
        else:
            threading.Timer(0.5, processImage, args=(frame,)).start()
                    
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            
def processImage(frame):
    image = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
    image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)
    image = (image / 127.5) - 1

    prediction = model.predict(image, verbose=0)
    index = np.argmax(prediction)
    class_name = str((class_names[index])[2:]).replace("\n", "")
    confidence_score = prediction[0][index]

    if class_name != "background":
        if sentence[-1] != class_name:
            sentence.append(class_name)
            # if sentence[-1] != ".":
            thread = threading.Thread(target=say, args=(class_name,))
            thread.start()

def say(text):
    speak = pyttsx3.init()
    speak.say(text)
    speak.runAndWait()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/data')
def sentence_feed():
    data = (' '.join(sentence))[1:]
    return jsonify(data=data)

if __name__ == "__main__":
    app.run(debug=True)
